1. YAEL

### Student Mental Health Analysis - Final Project
Authors: Eden Elkoubi, Yael Barbash, Avigail Cohen

Dataset: Student Mental Health Survey
Project Overview & Kick-off
This project investigates the relationship between academic disciplines (STEM vs. Non-STEM), social support, and mental health outcomes among students.

Research Questions
1.Is there a significant difference in anxiety and stress levels between STEM and Humanities/Social Science students?    
2.Does social support act as a moderator in the relationship between academic stress and mental health?

In [ ]:
code

2.AVIGAIL

In [ ]:
code

3.EDEN

### Setup and Logging 
 According to the project guidelines, we must use a logger instead of print statements to track the execution flow. This part sets up the professional logging configuration that will record all statistical results into a file named analysis_log.log.

In [ ]:
import pandas as pd
import numpy as np
import logging
from scipy import stats
import statsmodels.api as sm
##
# Configure the logger to track the analysis process
def setup_logger():
    """
    Sets up a logger that outputs to both a log file and the console.
    This replaces standard 'print' statements as per project requirements.
    """
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler("analysis_log.log"),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(__name__)
   
logger = setup_logger()
logger.info("Logger initialized. Ready for statistical analysis.")

### Hypothesis Testing - Independent T-Test
Explanation: The first hypothesis explores whether there is a significant difference in mental health scores between STEM students and students from other faculties. We use an Independent T-test, ensuring we handle missing values (NaN) correctly.

In [ ]:
def perform_group_comparison(df, group_col, target_col):
    """
    Hypothesis: Mental health levels differ between STEM and Non-STEM students.
    Method: Independent Two-Sample T-Test.
    """
    logger.info(f"Comparing {target_col} between groups in {group_col}")

    # Separating groups based on the binary indicator (1 for STEM, 0 for others)
    # Using meaningful variable names instead of raw indices
    stem_group = df[df[group_col] == 1][target_col]
    non_stem_group = df[df[group_col] == 0][target_col]

    # Performing the T-test
    t_stat, p_val = stats.ttest_ind(stem_group, non_stem_group, nan_policy='omit')

    logger.info(f"T-test results - Statistic: {t_stat:.4f}, P-value: {p_val:.4f}")
    
    # Interpretation logic
    if p_val < 0.05:
        logger.info("Result is statistically significant (p < 0.05).")
    else:
        logger.info("Result is not statistically significant (p >= 0.05).")
        
    return t_stat, p_val

### Correlation Analysis
Explanation: The second hypothesis checks for a linear relationship between academic performance (CGPA) and substance use. We use Pearson’s Correlation coefficient to determine the strength and direction of this relationship.

In [ ]:
def analyze_variable_correlation(df, var_x, var_y):
    """
    Hypothesis: There is a correlation between Academic Load/CGPA and Substance Use.
    Method: Pearson Correlation.
    """
    logger.info(f"Calculating correlation between {var_x} and {var_y}")

    # Cleaning data locally for this specific test
    valid_data = df[[var_x, var_y]].dropna()
    
    correlation_coeff, p_value = stats.pearsonr(valid_data[var_x], valid_data[var_y])

    logger.info(f"Correlation results - Coefficient: {correlation_coeff:.4f}, P-value: {p_value:.4f}")
    return correlation_coeff, p_value

### Moderation Analysis (Regression)
Explanation: To increase the "Complexity Level" (as required in the grading rubric), we implement a moderation model. We test if "Social Support" moderates the relationship between "Academic Load" and "Mental Health Stress".

In [ ]:
def run_moderation_model(df, outcome, predictor, moderator):
    """
    Hypothesis: Social Support acts as a moderator for the effect of Academic Load on Stress.
    Method: OLS Regression with an interaction term.
    """
    logger.info("Starting Moderation Analysis (Regression with interaction term)")

    # Creating the interaction term (Predictor * Moderator)
    df_temp = df.copy()
    df_temp['interaction'] = df_temp[predictor] * df_temp[moderator]

    # Independent variables: Predictor, Moderator, and their Interaction
    X = df_temp[[predictor, moderator, 'interaction']]
    X = sm.add_constant(X)  # Adds the intercept (constant) to the model
    y = df_temp[outcome]

    # Fit the OLS model
    model = sm.OLS(y, X, missing='drop').fit()

    logger.info("Moderation model fitting complete.")
    # We return the summary which contains all statistical details
    return model.summary()

### Main Execution Block
Explanation: This is the entry point of the script. Following the guidelines, the main() function is kept minimal, serving only to orchestrate the flow of the analysis modules.

In [ ]:
def main():
    """
    Main execution flow. In a real scenario, 'df' comes from the Preprocessing module.
    """
    logger.info("Final Project - Part II: Statistical Analysis Phase Started")

    # Placeholder: In your project, replace this with your actual cleaned DataFrame
    # df = pd.read_csv("your_cleaned_data.csv")

    # Example calls for testing the logic:
    # perform_group_comparison(df, 'Is_STEM', 'Stress_Score')
    # analyze_variable_correlation(df, 'CGPA', 'Substance_Use')
    # regression_results = run_moderation_model(df, 'Stress_Score', 'Academic_Load', 'Social_Support')
    
    logger.info("Statistical Analysis Phase Completed Successfully.")

if __name__ == "__main__":
    main()

### Data Visualization - Mental Health Group Comparison
To visually support the results of our Independent T-Test, we use a Box Plot. This visualization displays the distribution of mental health scores between STEM and Non-STEM students, highlighting the median, quartiles, and any potential outliers. This helps in understanding the variance and range of scores across different faculties.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_group_comparison(df, group_col, target_col):
    """
    Visualizes the difference between two academic groups using a box plot.
    This helps to interpret the variance found during the T-test.
    """
    logger.info(f"Generating Box Plot for {target_col} by {group_col}")
    
    # Initialize the plot style
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(10, 6))
    
    # Create Boxplot
    sns.boxplot(x=group_col, y=target_col, data=df, palette="Set2")
    
    # Add descriptive labels
    plt.title('Comparison of Mental Health Scores: STEM vs. Others', fontsize=14)
    plt.xlabel('Is STEM Student? (0 = No, 1 = Yes)', fontsize=12)
    plt.ylabel('Mental Health Score', fontsize=12)
    
    # Save the figure as required by best practices
    plt.savefig('visuals/group_comparison_boxplot.png')
    plt.show()
    
    logger.info("Group comparison plot saved successfully.")

### Data Visualization - Academic and Substance Correlation
Following the Correlation Analysis, we create a Scatter Plot with a Regression Line. This visualization helps us identify the relationship between Academic Performance (CGPA) and Substance Use. The regression line indicates the direction (positive/negative) and strength of the linear association between these two continuous variables.

In [ ]:
def plot_correlation_regplot(df, var_x, var_y):
    """
    Generates a scatter plot with a regression line to visualize 
    the relationship between Academic performance and Substance use.
    """
    logger.info(f"Generating Correlation Plot: {var_x} vs {var_y}")
    
    plt.figure(figsize=(10, 6))
    
    # Using regplot to visualize the trend line and confidence interval
    sns.regplot(x=var_x, y=var_y, data=df, 
                scatter_kws={'alpha': 0.5}, line_kws={'color': 'darkred'})
    
    plt.title(f'Relationship Trend: {var_x} and {var_y}', fontsize=14)
    plt.xlabel('Cumulative GPA (CGPA)', fontsize=12)
    plt.ylabel('Frequency of Substance Use', fontsize=12)
    
    plt.savefig('visuals/correlation_scatter_plot.png')
    plt.show()
    
    logger.info("Correlation plot saved successfully.")

### Data Visualization - Moderation Effect Interaction
The most complex part of our analysis is the Moderation Effect. This visualization demonstrates how Social Support moderates the impact of Academic Load on Mental Health Stress. By splitting the "Social Support" into high and low categories, we can see if the slope of the relationship changes, providing a clear visual representation of the interaction effect.

In [ ]:
def plot_moderation_interaction(df, outcome, predictor, moderator):
    """
    Visualizes the interaction effect between Academic Load and Social Support.
    This shows if Social Support acts as a buffer against academic stress.
    """
    logger.info("Generating Moderation Interaction Visualization")
    
    # Create a copy to prevent modifying the main dataframe
    df_plot = df.copy()
    
    # Categorize the moderator for visual clarity (High vs Low Support)
    median_val = df_plot[moderator].median()
    df_plot['Support_Level'] = df_plot[moderator].apply(
        lambda x: 'High Support' if x > median_val else 'Low Support'
    )
    
    # Plotting separate regression lines for each support level
    plt.figure(figsize=(10, 6))
    sns.lmplot(x=predictor, y=outcome, hue='Support_Level', data=df_plot, 
               aspect=1.5, height=6, palette='husl')
    
    plt.title('Interaction Analysis: Impact of Social Support on Academic Stress', fontsize=14)
    plt.xlabel('Academic Workload', fontsize=12)
    plt.ylabel('Mental Stress Level', fontsize=12)
    
    plt.savefig('visuals/moderation_interaction_plot.png')
    plt.show()
    
    logger.info("Moderation interaction plot saved successfully.")